In [2]:
import pandas as pd

# import numpy as np
import os

os.chdir("/Users/daniel.lopes/Projects/personal/ModelCrafter")
from crafter.models.linear_regression import LinearRegression as OLS
import statsmodels.api as sm
from crafter.utils.data_split import split_data

# Prepare data
data = pd.read_csv("data/raw/prostate-cancer.txt").drop(columns="id")
# X_train, X_test, y_train, y_test = split_data(data=data.drop(columns = "train"), target_column='lcavol', split_method='random', test_size=0.15 ,random_state=42)

# Splitting the data into training and test sets based on the 'train' column
train_data = data[data["train"] == "T"]
test_data = data[data["train"] != "T"]

# Dropping the 'train' and 'lpsa' columns for the feature sets
X_train = train_data.drop(columns=["train", "lpsa"])
X_test = test_data.drop(columns=["train", "lpsa"])

# Keeping only the 'lpsa' column for the target sets
y_train = train_data[["lpsa"]]
y_test = test_data[["lpsa"]]

In [3]:
model_crafter = OLS(fit_intercept=True)
model_crafter.fit(X_train, y_train)

# Print the Model Crafter summary
print("Model Crafter Summary:")
model_crafter.summary()

Model Crafter Summary:
Feature              | Coefficients    | Std Err    | z          | P>|z|      | [0.025    | 0.975]    
Intercept            | 0.4292          | 1.4455     | 0.2969     | 0.7676     | [-2.4040, 3.2623]
lcavol               | 0.5765          | 0.1000     | 5.7676     | 0.0000     | [0.3806, 0.7725]
lweight              | 0.6140          | 0.2077     | 2.9565     | 0.0045     | [0.2070, 1.0211]
age                  | -0.0190         | 0.0127     | -1.5003    | 0.1390     | [-0.0438, 0.0058]
lbph                 | 0.1448          | 0.0656     | 2.2096     | 0.0311     | [0.0164, 0.2733]
svi                  | 0.7372          | 0.2778     | 2.6539     | 0.0103     | [0.1928, 1.2817]
lcp                  | -0.2063         | 0.1028     | -2.0065    | 0.0495     | [-0.4079, -0.0048]
gleason              | -0.0295         | 0.1871     | -0.1577    | 0.8753     | [-0.3963, 0.3373]
pgg45                | 0.0095          | 0.0051     | 1.8678     | 0.0668     | [-0.0005, 0.0

In [4]:
# Add an intercept term to X_train for statsmodels
X_train_sm = sm.add_constant(X_train)

# Fit the model using statsmodels
model_sm = sm.OLS(y_train, X_train_sm).fit()

# Print the statsmodels summary
print("\nStatsmodels Summary:")
print(model_sm.summary())


Statsmodels Summary:
                            OLS Regression Results                            
Dep. Variable:                   lpsa   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     16.47
Date:                Wed, 14 Feb 2024   Prob (F-statistic):           2.04e-12
Time:                        10:59:57   Log-Likelihood:                -67.505
No. Observations:                  67   AIC:                             153.0
Df Residuals:                      58   BIC:                             172.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4292      1.5